# 🏈 QEPC NFL Quantum Prediction Model

A quantum-inspired NFL prediction model featuring:

- **Superposition**: Teams exist in multiple performance states
- **Entanglement**: Correlated offensive/defensive performances  
- **Interference**: Non-linear matchup effects
- **Tunneling**: Upset probability floors ("any given Sunday")
- **Decoherence**: Weather and environmental factors

## How It Works

1. Calculate team strengths from historical data
2. Each team has a quantum state distribution (DOMINANT → STRUGGLING)
3. Simulations "collapse" each team into a specific state
4. Drive-by-drive simulation captures NFL's discrete scoring

In [ ]:
# Setup
import sys
from pathlib import Path

# Add current folder to path
sys.path.insert(0, str(Path.cwd()))

import pandas as pd
import numpy as np

from nfl_quantum_engine import (
    NFLQuantumEngine, NFLTeamStrength, NFLQuantumConfig,
    QuantumState, TeamState, create_sample_teams
)
from nfl_strengths import NFLStrengthCalculator, generate_sample_nfl_games

print("✅ NFL Quantum Engine loaded!")

In [ ]:
# Generate sample data (or use your own)
sample_games = generate_sample_nfl_games(n_weeks=10)
sample_games.to_csv("sample_nfl_games.csv", index=False)
print(f"Created {len(sample_games)} sample games")
display(sample_games.head(10))

In [ ]:
# Calculate team strengths
calc = NFLStrengthCalculator()
calc.load_games("sample_nfl_games.csv")
strengths = calc.calculate_all_strengths()

print("\nPower Rankings:")
display(calc.power_rankings())

In [ ]:
# Initialize quantum engine
engine = NFLQuantumEngine()

for _, row in strengths.iterrows():
    engine.add_team(NFLTeamStrength(
        team=row['team'],
        off_efficiency=row['off_efficiency'],
        def_efficiency=row['def_efficiency'],
        momentum=row.get('momentum', 0.0),
    ))

print(f"Loaded {len(engine.teams)} teams")

In [ ]:
# Single game prediction
teams = list(engine.teams.keys())
HOME, AWAY = teams[0], teams[1]

pred = engine.predict_game(HOME, AWAY, n_simulations=10000)

print(f"🏈 {AWAY} @ {HOME}")
print(f"\nHome Win: {pred['home_win_prob']:.1%}")
print(f"Spread: {pred['predicted_spread']:+.1f}")
print(f"Total: {pred['predicted_total']:.1f}")
print(f"\nQuantum States:")
for state, prob in pred['state_distribution'].items():
    print(f"  {state}: {prob:.1%}")

In [ ]:
# Spread analysis
spreads = [-7, -3.5, -3, 0, 3, 3.5, 7]
print(f"\nSpread Analysis: {AWAY} @ {HOME}")
print("-" * 50)
for sp in spreads:
    home_cov, _ = engine.spread_probability(HOME, AWAY, sp, n_simulations=3000)
    print(f"  {sp:+.1f}: Home covers {home_cov:.1%}")

In [ ]:
# Weather impact
print(f"\nWeather Impact on Total:")
for weather in ['clear', 'rain', 'snow', 'wind']:
    p = engine.predict_game(HOME, AWAY, weather=weather, n_simulations=3000)
    print(f"  {weather:6}: {p['predicted_total']:.1f} pts")

In [ ]:
# Run backtest
from nfl_backtest_engine import NFLBacktestEngine

backtest = NFLBacktestEngine("sample_nfl_games.csv")
backtest.config.N_SIMULATIONS = 2000

results = backtest.run_backtest("2024-10-01", "2024-12-01")
if results:
    backtest.print_summary()